In [1]:
] activate .

  Activating project at `~/tmp/Tommy`


In [2]:
using Revise, Oscar, Interact

[ Info: Precompiling Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13]


#I  The Julia package 'Downloads' cannot be loaded.


WebIO._IJuliaInit()

In [3]:
QQx, x = QQ[:x];

In [4]:
k, a = number_field(x^4 - 12*x^3 + 36*x^2 - 36*x + 9);

In [5]:
galois_group(k)

(Permutation group of degree 4 and order 4, Galois context for x^4 - 12*x^3 + 36*x^2 - 36*x + 9 and prime 11)

The computation of galois groups uses p-adic approximations of the roots as default, here a splitting field over Q_11.To obtain more information about the group, one can use `describe`:

In [6]:
describe(ans[1])

"C2 x C2"

So there should be 3 quadratic subfields:

In [7]:
subfields(k, degree = 2)

3-element Vector{Tuple{AbsSimpleNumField, NumFieldHom{AbsSimpleNumField, AbsSimpleNumField, Hecke.MapDataFromAnticNumberField{AbsSimpleNumFieldElem}, Hecke.MapDataFromAnticNumberField{AbsSimpleNumFieldElem}, AbsSimpleNumFieldElem}}}:
 (Number field of degree 2 over QQ, Map: number field -> k)
 (Number field of degree 2 over QQ, Map: number field -> k)
 (Number field of degree 2 over QQ, Map: number field -> k)

To identify them, we'll compute their discriminants...

In [8]:
[discriminant(maximal_order(x[1])) for x = ans]

3-element Vector{ZZRingElem}:
 12
 8
 24

Next we want to study (quadratic) extensions. We start by creating some (parametrized) abelian extension, here the maximal elementary abelian extension of exponent 2 ramifed only at 2 and 3:

In [9]:
a = ray_class_field(8*3*maximal_order(k), n_quo = 2)

Class field defined mod (<24, 24>, InfPlc{AbsSimpleNumField, AbsSimpleNumFieldEmbedding}[]) of structure (Z/2)^2

(The `n_quo = 2` is just helping Oscar to avoid the discrete logarithms in finite fields). Next we compute all subfields and select the normal ones:

In [10]:
a = filter(is_normal, subfields(a, degree = 2));

In [11]:
a[1]

Class field defined mod (<24, 24>, InfPlc{AbsSimpleNumField, AbsSimpleNumFieldEmbedding}[]) of structure Z/2

A class field is paramterized by a quotient of some ray class group. At this point, the no defining equation of the field has been computed. To compute the galois group (over Q), we use class field theory:
 - the group is an extension: `1 -> C_2 -> X -> C2 x C2 -> 1`
 - thus an element of `H^2(C2 x C2, C2)`
 - galois cohomology gives `H^2(C2 x C2, C) = C_4` where `C` denotes the idel class group.
 - there is a canonical generator `delta` for this
 - the idel class group projects onto any ray class group
 - `X` as an element of the `H^2` is obtained by projecting `delta` down

In [12]:
b = [galois_group(x, QQ) for x = a];

In [13]:
b[1]

(Permutation group of degree 6 and order 8, (2-cochain with values in Z/2, G-module for permutation group of degree 4 and order 4 acting on Z/2))

In [14]:
[describe(x[1]) for x = b]

3-element Vector{String}:
 "C4 x C2"
 "Q8"
 "D8"

Lets look at this in more detail. We need the (a) G-module representing the idel-class group:

In [15]:
I = idel_class_gmodule(k)

Idel-group for Number field of degree 4 over QQ using ZZRingElem[2, 3] as places

Computing the cohomology is easy...

In [16]:
h2, mp, _ = cohomology_group(I, 2);

In [17]:
ngens(h2)

151

To see the structure, we compute the elementary divisors: the group should be cyclic of order 4:

In [18]:
s, ms = snf(h2)

(Z/4, Map: s -> h2)

A generator

In [19]:
a = mp(ms(s[1]))

2-cochain with values in (Z/2)^4 x (Z/8)^2 x (Z/16)^2 x Z/128 x Z^(17)

In [20]:
G, mG = automorphism_group(PermGroup, k)

(Permutation group of degree 4 and order 4, Map: G -> generic group of order 4 with multiplication table -> set of automorphisms of k)

In [21]:
i = a(G[1], G[1])

Abelian group element [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 8, 20, 0, 0, 126, -12, 20, 0, 0, 126, 20, 0, 0, 0, 0, 0]

... but we can compute an ideal that would represent the same class - in a ray class group modulo 12:

In [22]:
ideal(I, ans, coprime = 12*maximal_order(k))

1//64393976339170667658682988360825531103035565459171049081 * <64393976339170667658682988360825531103035565459171049081, 1515991503139397324064686911796474562595237736714534407128777737357218788646319737437810684103014930735855490609*_a^3 + 3409962610950941777134408565615699187469878399743563952865165457148798443739037460374752969637495248222417596290*_a^2 + 3062489011179093014930945748096191656862663843420271667183222662619108076034529877545494726599643413618694572786*_a + 995859769783882421604813159645782624801994185939177453223563263329762137475845807431783929950338242303795797653>
Norm: 267015044140013442019000207316647969600923241522278428496852902232973829647073916781627721438027691171807291536478260862716746640132905480235604130808475025290200998441
basis_matrix 
[64393976339170667658682988360825531103035565459171049081 0 0 0; 0 64393976339170667658682988360825531103035565459171049081 0 0; 0 0 64393976339170667658682988360825531103035565459171049081 0; 81499450997545043272501154

In [23]:
Oscar.GaloisCohomology_Mod.idel_finite(I, i)

Dict{AbsSimpleNumFieldOrderIdeal, Tuple{AbsSimpleNumFieldElem, Hecke.LocalFieldElem}} with 2 entries:
  <2, _a + 1>                        => (480754399878*_a^3 + 1540572726606*_a^2…
  <3, 8//3*_a^3 + 4*_a^2 + 3*_a + 6> => (1, 1)

In [24]:
collect(values(ans))[1]

(480754399878*_a^3 + 1540572726606*_a^2 + 1942720414463*_a + 925632589508, (2^1 + 2^2 + 2^7 + 2^9 + 2^11 + 2^12 + 2^13 + 2^14 + 2^15 + 2^16 + 2^20 + 2^21 + 2^24 + 2^25 + 2^26 + 2^27 + 2^29 + 2^30 + 2^31 + 2^32 + 2^33 + 2^34 + 2^35 + 2^37 + 2^38 + O(2^40))*a^3 + (2^2 + 2^3 + 2^4 + 2^5 + 2^7 + 2^8 + 2^10 + 2^12 + 2^14 + 2^18 + 2^20 + 2^21 + 2^23 + 2^24 + 2^25 + 2^29 + 2^30 + 2^31 + 2^33 + 2^34 + 2^36 + O(2^40))*a^2 + (2^0 + 2^2 + 2^4 + 2^5 + 2^6 + 2^7 + 2^8 + 2^9 + 2^10 + 2^13 + 2^14 + 2^17 + 2^19 + 2^21 + 2^25 + 2^26 + 2^27 + 2^28 + 2^29 + 2^31 + 2^33 + 2^34 + 2^38 + O(2^40))*a + 2^0 + 2^2 + 2^3 + 2^7 + 2^9 + 2^10 + 2^12 + 2^13 + 2^15 + 2^16 + 2^17 + 2^18 + 2^19 + 2^21 + 2^22 + 2^23 + 2^25 + 2^28 + 2^29 + 2^30 + 2^31 + 2^32 + 2^35 + O(2^40))

The 1st part is an element in `k` that has the 2nd element as value in the completion. 